<a href="https://colab.research.google.com/github/Ibtisam-a/Project-Implementation/blob/master/1_Kronecker_Product.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1)Kronecker Product**
 
# Sparse.linalg.spsolve from the SciPy library.

Rewrite  TU + UT = F as

$\mathcal{T}u = f$

where $\mathcal{T} = I \otimes T + T \otimes I$, $u = \text{vec}(U)$ and $f = \text{vec}(F)$

In [1]:
import numpy as np
from scipy.sparse import diags

#Our parameters
#number of internal nodes , unknowns, in each direction

N = 190  # our values: 190, 380, 760, 1520 and 3040

#number of internal nodes, unknowns, in each direction
n = N-2 
#step size 
h = 1/(n+1) 

#Matrix with zeros
U = np.zeros([n,n])

# x and y arrays between 0 and 1, spaced internal points
x = np.linspace(h, 1-h, n)
y = np.linspace(h, 1-h, n)

#Internal mesh without boundaries
X, Y = np.meshgrid(x, y, indexing='ij')

#our function 
F = np.sin(2*np.pi*X)*np.sin(2*np.pi*Y)  

#Tridiagonal matrix A
diagonals = [[1.5],[-0.5],[-0.5]]
A = np.multiply(1, diags(diagonals, [0, -1, 1], shape=(n, n)).toarray())

#Compute the exact solution on a specific time (0.00000000001) to be comapred 
Exact_sol = np.exp(-8*(np.pi*np.pi)*(0.00000000001))*np.sin(2*np.pi*X)*np.sin(2*np.pi*Y)



In [2]:

import numpy as np
import time
from scipy.sparse.linalg import spsolve
from scipy.sparse import kron
from scipy.linalg import solve


def kronecker_prodoct_solver(A,F):   
    n = len(F)
    start_time = time.time()
    F = np.reshape(F,-1)
    # T = I ⊗ T + T^* ⊗ I  \\  T^* point out the conjugat transpose, I is the identity matrix
    T = kron(np.eye(n),A) + kron(A.transpose(),np.eye(n)) 
    #Kronecker exexution time 
    kronecker_time = time.time() - start_time
    #Scipy sparse solver
    U = spsolve(T, F) 
    
    #Calculate elapsed time 
    end_time = time.time()
    solve_time = end_time - kronecker_time - start_time
    total_time = end_time - start_time
    all_timess = [kronecker_time, solve_time, total_time]

    #Reshape U to compute and plot error
    U = np.reshape(U, (n,n)) 
    return U, all_timess

In [ ]:
import numpy as np
def compute_error(U,Exact_sol):
    n = len(U)
    # The maximum difference between both solutions
    err_max_difference = 0  
    # The average difference between both solutions 
    err_ave_difference = 0

#For loop to compute the error
    for i in range(0,n):
        for j in range(0,n):
            err_ave_difference += np.absolute(Exact_sol[i][j] - U[i][j])**2
            if np.absolute(Exact_sol[i][j] - U[i][j]) > err_max_difference:
                err_max_difference = np.absolute(Exact_sol[i][j] - U[i][j])       
    err_ave_difference = (err_ave_difference * h**2)**0.5
    return err_max_difference, err_ave_difference

#Solving our system
U, all_timess = kronecker_prodoct_solver(A,F)
err_max_difference, err_ave_difference = compute_error(U, Exact_sol)

print('Time taken:', all_timess)
print('Maximum diffence:', err_max_difference, '\n', 'Average difference:', err_ave_difference)
